# Unit 2B Portfolio Project (Side Notebook Strictly for Data Cleaning and Refining)
See [Full Version](https://colab.research.google.com/drive/1s3SJs2dpnH2LQvR9S3JNH2C-yD1na_4R) for Predictive Models & Visualization

UNIT 2B PORTFOLIO PROJECT COMPLETED BY: [RYAN ZERNACH](http://ryan.zernach.com/portfolio/)

![Ryan Zernach](https://ryan.zernach.com/wp-content/uploads/brizy/503/assets/images/iW=440&iH=440&oX=0&oY=0&cW=440&cH=440/Ryan_Zernach_The_Zernach_Foundation_0.png)

### Air Flights Datasets Courtesy of Department of Transportation's [Bureau of Transportation Statistics](https://www.transtats.bts.gov/Tables.asp?DB_ID=125&DB_Name=Airline%20Origin%20and%20Destination%20Survey%20%28DB1B%29&DB_Short_Name=Origin%20and%20Destination%20Survey) (once link is clicked, select "DB1BMarket") — Data is from 2018 flights in Q1, Q2, Q3, and Q4 that originate somewhere in the USA and have a destination of somewhere in the USA (USA includes Alaska, Hawaii, and US territories)

# NOTE: This code will not run all the way through because the datasets are too large for colab, so I had to breakdown the dataset into more manageable chunks, reduce/refine, concatenate, and do that again until I had one, clean dataset, which I [uploaded to Kaggle](https://www.kaggle.com/zernach/2018-airplane-flights) for others to use.

### 0.1) The data files were downloaded from the Department of Transportation and were then stored in my Google Drive. So the following lines of code request an authentication code from Google Drive, grant permissions, and package the Q1, Q2, Q3, and Q4 csv files into a format that's transferrable from Google Drive to Pandas dataframes. Then, do a bunch of cleaning/wrangling/refining and bundle them altogether into a single 2018 flights dataset.

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import pandas as pd

# Requests an authentication code, which is received by clicking the unique link
# that it provides when this code cell is run...
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Searches for all files that contain "Air Flight Information" and are within
# the Google Drive Folder with the long, seemingly-random extension shown in orange
listed = drive.ListFile({'q': "title contains 'Flight' and '14GnxygKyjd1N49QA8_ELzx8GnpbVRB2b' in parents"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

title Air Flight Information (2018, Q4).csv, id 1TJc8oh9t5Z0wzT4PN2JAxlBizwdFy1QI
title Air Flight Information (2018, Q3).csv, id 1jys2LPb0MSNrkeYdio-l-qr3-oMLNKoT
title Air Flight Information (2018, Q2).csv, id 1HHiazRI4c9ifgdKmESwEQdMBg6U5s_MB
title Air Flight Information (2018, Q1).csv, id 1SaeyCJYVLJEFhLbC4KYruMpnPgrm374v


In [ ]:
download_path = os.path.expanduser('~/data')
try:
  os.makedirs(download_path)
except FileExistsError:
  pass

# Using the 'title' and 'id' shown in the output two cells above, copy/paste
# that information into the following:

# Q1 Flights
output_file_q1 = os.path.join(download_path, 'Air Flight Information (2018, Q1).csv')
temp_file_q1 = drive.CreateFile({'id': '1SaeyCJYVLJEFhLbC4KYruMpnPgrm374v'})
temp_file_q1.GetContentFile(output_file_q1)

# Q2 Flights
output_file_q2 = os.path.join(download_path, 'Air Flight Information (2018, Q2).csv')
temp_file_q2 = drive.CreateFile({'id': '1HHiazRI4c9ifgdKmESwEQdMBg6U5s_MB'})
temp_file_q2.GetContentFile(output_file_q2)

# Q3 Flights
output_file_q3 = os.path.join(download_path, 'Air Flight Information (2018, Q3).csv')
temp_file_q3 = drive.CreateFile({'id': '1jys2LPb0MSNrkeYdio-l-qr3-oMLNKoT'})
temp_file_q3.GetContentFile(output_file_q3)

# Q4 Flights
output_file_q4 = os.path.join(download_path, 'Air Flight Information (2018, Q4).csv')
temp_file_q4 = drive.CreateFile({'id': '1TJc8oh9t5Z0wzT4PN2JAxlBizwdFy1QI'})
temp_file_q4.GetContentFile(output_file_q4)

# Q1 & Q2 Cleaned Flights
output_file_q_1_2 = os.path.join(download_path, 'q1_q2 Flights .csv')
temp_file_q_1_2 = drive.CreateFile({'id': '1Vuc4Dwd-1PnYZ_iFcJ2svG8ikBwDDbtj'})
temp_file_q_1_2.GetContentFile(output_file_q_1_2)

# Q3 & Q4 Cleaned Flights
output_file_q_3_4 = os.path.join(download_path, 'q3_q4 Flights .csv')
temp_file_q_3_4 = drive.CreateFile({'id': '1N1A_i7UGb-mhLNkA1m7FVyKCJKR2ruVQ'})
temp_file_q_3_4.GetContentFile(output_file_q_3_4)

#flights = pd.concat([pd.read_csv(output_file_q_1_2), pd.read_csv(output_file_q_3_4)])
flights = pd.concat([pd.read_csv(output_file_q3), pd.read_csv(output_file_q4)])


In [ ]:
# Drop the unnecessary/duplicate columns
unnecessary_columns = ['Year', 'OriginAirportID',
                       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCountry',
                       'OriginStateFips', 'OriginState', 'OriginStateName',
                       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 
                       'DestCountry', 'DestStateFips', 'DestState', 'DestStateName', 
                       'AirportGroup', 'WacGroup', 'TkCarrierGroup', 'OpCarrierGroup',
                       'MktDistanceGroup', 'ItinGeoType', 'Unnamed: 41']
flights = flights.drop(unnecessary_columns, axis=1)

# Use only the data where customer did not switch carrier, then drop the column
flights = flights[flights['TkCarrierChange'] == 0]
flights = flights.drop(['TkCarrierChange'], axis=1)

# Use only the data where customer did not switch operating company, then drop the column
flights = flights[flights['OpCarrierChange'] == 0]
flights = flights.drop(['OpCarrierChange'], axis=1)

# Use only the data where customer experience didn't change from one company to another, then drop two columns
flights = flights[(flights['RPCarrier'] == flights['TkCarrier']) & (flights['RPCarrier'] == flights['OpCarrier'])]
flights = flights.drop(['RPCarrier', 'OpCarrier'], axis=1)

# Use only the data where # miles flown is equal to # nonstop miles then drop one
flights = flights[flights['MktMilesFlown'] == flights['NonStopMiles']]
flights = flights.drop(['NonStopMiles'], axis=1)

# Use only the data where distance equals miles, then drop one of the columns
flights = flights[(flights['MktDistance'] == flights['MktMilesFlown'])]
flights = flights.drop(['MktDistance'], axis=1)

# Eliminate the upper-bound outlier prices and the ultra-cheap discounted coupon prices
flights = flights[(flights['MktFare'] >= 50) & (flights['MktFare'] <= 1000)]

# Eliminate the airlines that have been reduced to just a few tickets sold due to the above refinement processflights
flights = flights[(flights['TkCarrier'] != 'QX') & (flights['TkCarrier'] != 'OO') & (flights['TkCarrier'] != 'EV')]

# Drop the BulkFare column because it has no variety after all the above refining
flights = flights.drop(['BulkFare'], axis=1)

# Refine the dataset so the user cannot select more than 20 passengers for their purchase
flights = flights[flights['Passengers'] <= 20]

flights

,ItinID,MktID,MktCoupons,Quarter,Origin,OriginWac,Dest,DestWac,TkCarrier,Passengers,MktFare,MktMilesFlown,MktGeoType
73,20183294427,2018329442704,1,3,LAX,91,CMH,44,AA,1.0,162.93,1995.0,2
75,20183294428,2018329442804,1,3,LAX,91,CMH,44,AA,1.0,270.45,1995.0,2
132,20183294467,2018329446703,1,3,LAX,91,CMH,44,AA,1.0,137.77,1995.0,2
134,20183294468,2018329446803,1,3,LAX,91,CMH,44,AA,1.0,188.65,1995.0,2
136,20183294469,2018329446903,1,3,LAX,91,CMH,44,AA,1.0,203.05,1995.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7176132,201844999082,20184499908202,1,4,STL,64,AUS,74,WN,1.0,310.10,721.0,2
7176145,201844999092,20184499909203,1,4,BNA,54,MCI,64,WN,1.0,106.17,491.0,2
7176177,201844999112,20184499911203,1,4,BOS,13,LIT,71,WN,1.0,161.19,1260.0,2
7176179,201844999113,20184499911303,1,4,BOS,13,LIT,71,WN,1.0,170.87,1260.0,2


In [ ]:
flights['Miles'] = flights['MktMilesFlown']
flights = flights.drop(['MktMilesFlown'], axis=1)

flights['ContiguousUSA'] = flights['MktGeoType']
flights = flights.drop(['MktGeoType'], axis=1)

flights['NumTicketsOrdered'] = flights['Passengers']
flights = flights.drop(['Passengers'], axis=1)

flights['AirlineCompany'] = flights['TkCarrier']
flights = flights.drop(['TkCarrier'], axis=1)

flights['PricePerTicket'] = flights['MktFare']
flights = flights.drop(['MktFare'], axis=1)

flights

,ItinID,MktID,MktCoupons,Quarter,Origin,OriginWac,Dest,DestWac,Miles,ContiguousUSA,NumTicketsOrdered,AirlineCompany,PricePerTicket
73,20183294427,2018329442704,1,3,LAX,91,CMH,44,1995.0,2,1.0,AA,162.93
75,20183294428,2018329442804,1,3,LAX,91,CMH,44,1995.0,2,1.0,AA,270.45
132,20183294467,2018329446703,1,3,LAX,91,CMH,44,1995.0,2,1.0,AA,137.77
134,20183294468,2018329446803,1,3,LAX,91,CMH,44,1995.0,2,1.0,AA,188.65
136,20183294469,2018329446903,1,3,LAX,91,CMH,44,1995.0,2,1.0,AA,203.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7176132,201844999082,20184499908202,1,4,STL,64,AUS,74,721.0,2,1.0,WN,310.10
7176145,201844999092,20184499909203,1,4,BNA,54,MCI,64,491.0,2,1.0,WN,106.17
7176177,201844999112,20184499911203,1,4,BOS,13,LIT,71,1260.0,2,1.0,WN,161.19
7176179,201844999113,20184499911303,1,4,BOS,13,LIT,71,1260.0,2,1.0,WN,170.87


In [ ]:
flights = flights.reset_index(drop=True)
flights

,ItinID,MktID,MktCoupons,Quarter,Origin,OriginWac,Dest,DestWac,Miles,ContiguousUSA,NumTicketsOrdered,AirlineCompany,PricePerTicket
0,20183294427,2018329442704,1,3,LAX,91,CMH,44,1995.0,2,1.0,AA,162.93
1,20183294428,2018329442804,1,3,LAX,91,CMH,44,1995.0,2,1.0,AA,270.45
2,20183294467,2018329446703,1,3,LAX,91,CMH,44,1995.0,2,1.0,AA,137.77
3,20183294468,2018329446803,1,3,LAX,91,CMH,44,1995.0,2,1.0,AA,188.65
4,20183294469,2018329446903,1,3,LAX,91,CMH,44,1995.0,2,1.0,AA,203.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4881170,201844999082,20184499908202,1,4,STL,64,AUS,74,721.0,2,1.0,WN,310.10
4881171,201844999092,20184499909203,1,4,BNA,54,MCI,64,491.0,2,1.0,WN,106.17
4881172,201844999112,20184499911203,1,4,BOS,13,LIT,71,1260.0,2,1.0,WN,161.19
4881173,201844999113,20184499911303,1,4,BOS,13,LIT,71,1260.0,2,1.0,WN,170.87


In [ ]:
flights.to_csv('q3_q4 Flights .csv')
#flights.to_csv('q3_q4 Flights .csv')

# COMPRESS INTO ONE FILE

In [ ]:
from google.colab import files
uploaded = files.upload()

import io
df2 = pd.read_csv(io.BytesIO(uploaded['Filename.csv']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/q1_q2 Flights .csv')

In [ ]:
df2 = pd.read_csv('/content/q3_q4 Flights .csv')

In [ ]:
flights = pd.concat([df, df2])
flights

,Unnamed: 0,ItinID,MktID,MktCoupons,Quarter,Origin,OriginWac,Dest,DestWac,Miles,ContiguousUSA,NumTicketsOrdered,AirlineCompany,PricePerTicket
0,0,20181767585,2018176758501,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,672.87
1,1,20181767586,2018176758601,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,367.68
2,2,20181767587,2018176758701,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,417.94
3,3,20181767636,2018176763601,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,247.10
4,4,20181767637,2018176763701,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,276.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4881170,4881170,201844999082,20184499908202,1,4,STL,64,AUS,74,721.0,2,1.0,WN,310.10
4881171,4881171,201844999092,20184499909203,1,4,BNA,54,MCI,64,491.0,2,1.0,WN,106.17
4881172,4881172,201844999112,20184499911203,1,4,BOS,13,LIT,71,1260.0,2,1.0,WN,161.19
4881173,4881173,201844999113,20184499911303,1,4,BOS,13,LIT,71,1260.0,2,1.0,WN,170.87


In [ ]:
flights = flights.reset_index(drop=True)
flights = flights.drop(['Unnamed: 0'], axis=1)
flights

,ItinID,MktID,MktCoupons,Quarter,Origin,OriginWac,Dest,DestWac,Miles,ContiguousUSA,NumTicketsOrdered,AirlineCompany,PricePerTicket
0,20181767585,2018176758501,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,672.87
1,20181767586,2018176758601,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,367.68
2,20181767587,2018176758701,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,417.94
3,20181767636,2018176763601,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,247.10
4,20181767637,2018176763701,1,1,PHL,23,LAX,91,2402.0,2,1.0,AA,276.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9534412,201844999082,20184499908202,1,4,STL,64,AUS,74,721.0,2,1.0,WN,310.10
9534413,201844999092,20184499909203,1,4,BNA,54,MCI,64,491.0,2,1.0,WN,106.17
9534414,201844999112,20184499911203,1,4,BOS,13,LIT,71,1260.0,2,1.0,WN,161.19
9534415,201844999113,20184499911303,1,4,BOS,13,LIT,71,1260.0,2,1.0,WN,170.87


In [ ]:
flights.to_csv('Cleaned_2018_Flights.csv')